<a href="https://colab.research.google.com/github/romerofernandes/Car_consumption/blob/main/Car_consumption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import matplotlib.pyplot as plt

In [32]:
pd.set_option('display.max_rows', None)

In [33]:
all_csv_files = glob.glob("/content/*.csv")
all_csv_files

['/content/focus.csv',
 '/content/skoda.csv',
 '/content/toyota.csv',
 '/content/cclass.csv',
 '/content/unclean cclass.csv',
 '/content/unclean focus.csv',
 '/content/audi.csv',
 '/content/vauxhall.csv',
 '/content/ford.csv',
 '/content/hyundi.csv',
 '/content/merc.csv',
 '/content/bmw.csv',
 '/content/vw.csv']

In [34]:
df_car = pd.concat(map(pd.read_csv, all_csv_files), axis=0, ignore_index=True)

In [35]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118150 entries, 0 to 118149
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   model         117995 non-null  object 
 1   year          117903 non-null  float64
 2   price         117995 non-null  object 
 3   transmission  117995 non-null  object 
 4   mileage       117077 non-null  object 
 5   fuelType      108540 non-null  object 
 6   engineSize    108540 non-null  float64
 7   tax           94327 non-null   float64
 8   mpg           99187 non-null   float64
 9   fuel type     3517 non-null    object 
 10  engine size   9345 non-null    object 
 11  mileage2      9399 non-null    object 
 12  fuel type2    8537 non-null    object 
 13  engine size2  8537 non-null    object 
 14  reference     9455 non-null    object 
 15  tax(£)        4860 non-null    float64
dtypes: float64(5), object(11)
memory usage: 14.4+ MB


#Transformações

In [36]:
df_car.dropna(subset=['mpg'], how='any', inplace=True)

In [37]:
df_car.dropna(axis='columns', inplace=True)

In [38]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 5454 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  float64
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   engineSize    99187 non-null  float64
 7   mpg           99187 non-null  float64
dtypes: float64(3), object(5)
memory usage: 6.8+ MB


##Combinando modelos

In [39]:
a = df_car.model.value_counts().reset_index()

In [40]:
a.columns = ['model', 'qtd']

In [41]:
finish = False
limit_s = 3000
limit_i = 2000

group_car = 1
find_start = 0
while a.qtd.iloc[find_start] > limit_i:
  find_start +=1

previous = find_start
actual = find_start + 2

while finish is False:
  if (a.qtd.iloc[previous:actual].sum() < limit_s) & (a.qtd.iloc[previous:actual].sum() >= limit_i):
    modelos = a.model.iloc[previous:actual].values
    
    for modelo in modelos:
      df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)
    
    group_car +=1
    previous = actual
    actual += 2
  else:
    if (a.qtd.iloc[previous:a.shape[0]].sum() < limit_s) & (a.qtd.iloc[previous:a.shape[0]].sum() >= limit_i):
      modelos = a.model.iloc[previous:a.shape[0]].values
      for modelo in modelos:
        df_car.model.replace(modelo, 'car' + str(group_car), inplace=True)
      finish=True

    elif a.qtd.iloc[previous:actual].sum() <= limit_i:
      actual +=1
    elif a.qtd.iloc[previous:actual].sum() > limit_s:
      previous +=1
      actual +=1
  
  if actual > a.shape[0]:
    finish=True

##Combinando anos

In [42]:
df_car.year = df_car.year.astype(int)
df_car.year = df_car.year.astype(str)

In [43]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99187 entries, 5454 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99187 non-null  object 
 1   year          99187 non-null  object 
 2   price         99187 non-null  object 
 3   transmission  99187 non-null  object 
 4   mileage       99187 non-null  object 
 5   fuelType      99187 non-null  object 
 6   engineSize    99187 non-null  float64
 7   mpg           99187 non-null  float64
dtypes: float64(2), object(6)
memory usage: 6.8+ MB


In [44]:
df_car.loc[df_car.year=='2060'].index

Int64Index([73994], dtype='int64')

In [45]:
df_car.drop(index=df_car.loc[df_car.year=='2060'].index, inplace=True)

In [46]:
b = df_car.year.value_counts().reset_index()

In [47]:
b.columns = ['year', 'qtd']

In [48]:
anos = b.year.iloc[4:].values
for ano in anos:
        df_car.year.replace(ano, 'outros', inplace=True)

In [49]:
df_car.year.value_counts()

2019      26771
2017      21733
outros    21168
2016      15850
2018      13664
Name: year, dtype: int64

##Combinando Engine Size

In [50]:
df_car.head()

,model,year,price,transmission,mileage,fuelType,engineSize,mpg
5454,car1,2017,10550,Manual,25250,Petrol,1.4,54.3
5455,car14,2018,8200,Manual,1264,Petrol,1.0,67.3
5456,car1,2019,15650,Automatic,6825,Diesel,2.0,67.3
5457,car14,outros,14000,Automatic,28431,Diesel,2.0,51.4
5458,car9,2019,18350,Manual,10912,Petrol,1.5,40.9


In [51]:
c = df_car.engineSize.value_counts().reset_index()

In [52]:
c.columns = ['engineSize', 'qtd']

In [53]:
c.qtd.iloc[7:].sum()

10939

In [54]:
engine_size = c.engineSize.iloc[7:].values
df_car.engineSize.replace(engine_size, 'outros', inplace=True)

In [55]:
c

,engineSize,qtd
0,2.0,27233
1,1.0,17323
2,1.5,11418
3,1.4,11026
4,1.6,8837
5,1.2,6963
6,3.0,5447
7,2.1,4018
8,1.8,1717
9,1.3,1372


In [59]:
c = df_car.engineSize.value_counts().reset_index()
c.columns = ['engineSize', 'qtd']

In [60]:
c

,engineSize,qtd
0,2.0,27233
1,1.0,17323
2,1.5,11418
3,1.4,11026
4,outros,10939
5,1.6,8837
6,1.2,6963
7,3.0,5447


In [62]:
c.qtd.iloc[6:].sum()

12410

In [64]:
df_car.engineSize.replace([1.6, 1.2], '1.2/1.6', inplace=True)

In [65]:
df_car.engineSize.replace(3.0, 'outros', inplace=True)

In [66]:
df_car.engineSize.value_counts()

2.0        27233
1.0        17323
outros     16386
1.2/1.6    15800
1.5        11418
1.4        11026
Name: engineSize, dtype: int64

#testes

In [70]:
df_car.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99186 entries, 5454 to 118149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         99186 non-null  object 
 1   year          99186 non-null  object 
 2   price         99186 non-null  float64
 3   transmission  99186 non-null  object 
 4   mileage       99186 non-null  float64
 5   fuelType      99186 non-null  object 
 6   engineSize    99186 non-null  object 
 7   mpg           99186 non-null  float64
dtypes: float64(3), object(5)
memory usage: 6.8+ MB


In [68]:
df_car.head()

,model,year,price,transmission,mileage,fuelType,engineSize,mpg
5454,car1,2017,10550,Manual,25250,Petrol,1.4,54.3
5455,car14,2018,8200,Manual,1264,Petrol,1.0,67.3
5456,car1,2019,15650,Automatic,6825,Diesel,2.0,67.3
5457,car14,outros,14000,Automatic,28431,Diesel,2.0,51.4
5458,car9,2019,18350,Manual,10912,Petrol,1.5,40.9


In [69]:
df_car.price = df_car.price.astype('float')
df_car.mileage = df_car.mileage.astype('float')